In [3]:
import re
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import countDistinct
from pyspark.sql.types import Row
from pyspark.sql import SQLContext
from pyspark.sql.utils import AnalysisException
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [2]:
def find_ext_links(entity, regex=r"\[\[(.*?)\]\]"):
    regex_compiled = re.compile(regex, re.IGNORECASE)
    text = entity.revision.text._VALUE
    refs = regex_compiled.findall(text)
    return Row(id=entity.id, title=entity.title, link_count=len(refs), link=refs)

In [ ]:
arg = "popularity_military_conflict" 
# "popularity_civil_conflict.parquet" "popularity_military_conflict.parquet" "popularity_civilian_attack.parquet"
args= {
    'in_filtered': "../../data/data_processed/{}.parquet".format(arg), 
    'in_raw': "./first.parquet/", 
    'out': '../../temp/{}.parquet'.format(arg) # TODO change
}

In [3]:
# just to test
args= {
    'in_filtered': "../../data/data_processed/wiki_civil_conflict_df.parquet", 
    'in_raw': "./10million.parquet/", 
    'out': './test_ext_links.parquet'
}

In [4]:
#new
# load filtered articles (parquet)
conflict_articles = spark.read.parquet(args['in_filtered'])
# load raw data (parquet)
wikipedia = spark.read.parquet(args['in_raw'])

# filter to select articles with text that are not redirections
# take articles with text without any redirection
try:
    articles = wikipedia.filter("ns = '0'").filter("redirect._title is null")\
        .filter("revision.text._VALUE is not null")\
        .filter("length(revision.text._VALUE) > 0")  # TODO this does not work locally but on cluster?
except AnalysisException:
    print('Not removing redirections because field redirect._title  not present in dataframe')
    articles = wikipedia.filter("ns = '0'").filter("revision.text._VALUE is not null")\
        .filter("length(revision.text._VALUE) > 0")

Not removing redirections because field redirect._title  not present in dataframe


In [5]:
external_links = sqlContext.createDataFrame(articles.rdd.map(find_ext_links))

In [7]:
all_info = conflict_articles.join(external_links, "id", how='inner')
#all_info = all_info.select("id", "title", "external_links")
all_info.write.parquet(args['out'])
all_info.show()

+-----+-----+--------------------+--------------------+--------------------+----------+------------------+
|   id|death|            end_date|            location|                link|link_count|             title|
+-----+-----+--------------------+--------------------+--------------------+----------+------------------+
|75829| null|+1893-00-00T00:00...|Powder River Coun...|[Fort D.A. Russel...|       145|Johnson County War|
|43573| null|      April 21, 1855|[[Chicago, Illino...|[bar|saloon, Germ...|        51|   Lager Beer Riot|
+-----+-----+--------------------+--------------------+--------------------+----------+------------------+



In [8]:
a = spark.read.parquet("./test_ext_links.parquet/")

In [9]:
a.printSchema()

root
 |-- id: integer (nullable = true)
 |-- death: string (nullable = true)
 |-- end_date: string (nullable = true)
 |-- location: string (nullable = true)
 |-- link: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- link_count: long (nullable = true)
 |-- title: string (nullable = true)



In [10]:
a.show()

+-----+-----+--------------------+--------------------+--------------------+----------+------------------+
|   id|death|            end_date|            location|                link|link_count|             title|
+-----+-----+--------------------+--------------------+--------------------+----------+------------------+
|75829| null|+1893-00-00T00:00...|Powder River Coun...|[Fort D.A. Russel...|       145|Johnson County War|
|43573| null|      April 21, 1855|[[Chicago, Illino...|[bar|saloon, Germ...|        51|   Lager Beer Riot|
+-----+-----+--------------------+--------------------+--------------------+----------+------------------+

